[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r-xue/casa_proc/blob/master/demo/test_casaproc.ipynb)

## Example BX610: perform model fitting using method='emcee'

In [31]:
import sys
import glob
import os
import io
import logging
import emcee
from pprint import pprint

print(sys.version)


import socket 
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir('/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output/')
print(socket.gethostname())
print(os.getcwd())

import gmake
print(gmake.__version__)
print(gmake.__email__)
print(gmake.__demo__)
gmake.check_deps()

#inpfile=gmake.__demo__+'/../examples/inpfile/bx610_b4c5_uv_mc.inp'
inpfile=gmake.__demo__+'/../examples/inpfile/bx610_b6c3_uv_mc.inp'
logfile=''

print('>'*40)
#gmake.logger_config()
#gmake.logger_status()
#import pprint
#pprint.pprint(logging.Logger.manager.loggerDict) 
gmake.logger_config()
inp_dct=gmake.read_inp(inpfile)
outdir=inp_dct['general']['outdir']
gmake.logger_config(logfile=outdir+'/gmake.log',loglevel='DEBUG',logfilelevel='DEBUG')
gmake.logger_status()


astropy            >=3.2.2      3.2.2       
emcee              >=3.0.0      3.0.0       
corner             >=2.0        2.0.1       
tqdm               unspecified  4.31.1      
lmfit              unspecified  0.9.12      
asteval            >=0.9.14     0.9.14      
numexpr            >=2.7.0      2.7.0       
hickle             unspecified  3.4.5       
alpy               unspecified  0.22.0      
regions            unspecified  0.3         
scipy              unspecified  1.2.1       
reproject          unspecified  0.6.dev644  
python-casacore    >=3.1.1      3.1.1       
scikit-image       unspecified  0.14.2      
galpy              unspecified  1.5.dev0    
mkl-fft            unspecified  1.0.14      
pvextractor        >=0.2.dev327 0.2.dev327  
spectral-cube      >=0.4.5.dev  0.4.5.dev2235
radio-beam         >=0.3        0.3.3.dev397
reproject          >=0.6.dev    0.6.dev644  
casa-proc          unspecified  0.1.dev3    


3.7.5 (default, Oct 19 2019, 11:15:26) 
[Clang 11.0.0 (clang-1100.0.33.8)]
hyperion
/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output
0.2.dev1
rx.astro@gmail.com
/Users/Rui/Dropbox/Worklib/projects/GMaKE/gmake
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<Logger gmake (DEBUG)>
[<FileHandler /Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output/bx610_b6c3_uv_mc/gmake.log (DEBUG)>, <StreamHandler stderr (DEBUG)>]


In [46]:
dat_dct=gmake.read_data(inp_dct,fill_mask=True,fill_error=True,save_data=True)

read data (may take some time..)

Read: ../data/bx610/alma/2015.1.00250.S/bb2.ms

data@../data/bx610/alma/2015.1.00250.S/bb2.ms                (56354, 238)         102 MiB             
uvw@../data/bx610/alma/2015.1.00250.S/bb2.ms                 (56354, 3)           661 KiB             
weight@../data/bx610/alma/2015.1.00250.S/bb2.ms              (56354,)             220 KiB             161.43835
chanfreq@../data/bx610/alma/2015.1.00250.S/bb2.ms            (238,)       250.7529 GHz   252.6043 GHz
chanwidth@../data/bx610/alma/2015.1.00250.S/bb2.ms           (238,)         7.8118 MHz     7.8118 MHz
phasecenter@../data/bx610/alma/2015.1.00250.S/bb2.ms         23h46m09.44s  12d49m19.3s
data flagging fraction: 0.0

Read: ../data/bx610/alma/2015.1.00250.S/bb3.ms

data@../data/bx610/alma/2015.1.00250.S/bb3.ms                (53466, 238)          97 MiB             
uvw@../data/bx610/alma/2015.1.00250.S/bb3.ms                 (53466, 3)           627 KiB             
weight@../data/bx610/alma/

In [47]:
inp_dct=gmake.read_inp(inpfile)
pprint(gmake.gmake_utils.inp2mod(inp_dct))

{'bx610-dynamics': {'disk_rs': 1.0, 'disk_sd': 5000000000.0, 'halo_mvir': 0.5},
 'ci21': {'inc': 44.06318834223217,
          'intflux': 0.6493000000000002,
          'method': 'disk3d',
          'note': 'CI21 of BX610 in BB2',
          'object': 'bx610',
          'pa': -52.40943192318819,
          'restfreq': 809.34197,
          'sbser': [0.1877150243974696, 1.0],
          'type': 'disk3d',
          'vdis': [50.82245315749322,
                   50.82245315749322,
                   50.82245315749322,
                   50.82245315749322,
                   50.82245315749322],
          'vis': '../data/bx610/alma/2015.1.00250.S/bb2.ms',
          'vrad': [0, 0.12, 0.24, 0.36, 0.48],
          'vrot': [0.0,
                   197.03794217978916,
                   197.03794217978916,
                   197.03794217978916,
                   197.03794217978916],
          'vsys': 117.49374310082439,
          'xypos': [356.53932590540563, 12.82201818705264],
          'z': 2.21},

In [58]:
from gmake import model_lnprob
fit_dct,sampler=gmake.fit_setup(inp_dct,dat_dct)
lnl,blobs=model_lnprob(fit_dct['p_start'],fit_dct,inp_dct,dat_dct,
                               savemodel=inp_dct['general']['outdir'],packblobs=True)

#print(type(blobs))
#print(np.shape(blobs))

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
optimizer: emcee
optimizing parameters: index / name / start / lo_limit / up_limit / scale
 0   vsys@bx610-basics          117.49      (    -120.00   ,     280.00     )     40.00     
 1   vrot[1:5]@bx610-basics     197.04      (     0.00     ,     800.00     )     40.00     
 2   vdis[0:5]@bx610-basics      50.82      (     0.00     ,     800.00     )     10.00     
 3   pa@bx610-basics            -52.41      (    -132.41   ,      27.59     )     5.00      
 4   inc@bx610-basics            44.06      (     5.00     ,      85.00     )     5.00      
 5   xypos[0]@bx610-basics    356.5393259   (  356.5390481 ,   356.5396037  )   0.0000389   
 6   xypos[1]@bx610-basics    12.8220182    (  12.8217404  ,   12.8222960   )   0.0000389   
 7   intflux@co76                1.30       (     0.10     ,     200.00     )     0.05      
 8   sbser[0]@co76               0.22       (     0.01     ,      1.00    

bx610_b6c3_uv_mc
['cube.', 'mfs.', 'cube3.']
[('../data/bx610/alma/2015.1.00250.S/', 'b6c3_')]


copy ms container: 
    ../data/bx610/alma/2015.1.00250.S/bb2.ms  to  bx610_b6c3_uv_mc/model_b6c3_bb2.ms
write ms column: 
    uvmodel@../data/bx610/alma/2015.1.00250.S/bb2.ms to data@bx610_b6c3_uv_mc/model_b6c3_bb2.ms
create symlink:
    /Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/bx610/alma/2015.1.00250.S/bb2.ms  to  bx610_b6c3_uv_mc/data_b6c3_bb2.ms
 
-->data_b6c3_bb3.ms
 
imod2d@../data/bx610/alma/2015.1.00250.S/bb3.ms
write reference model image: 
    imod2d@../data/bx610/alma/2015.1.00250.S/bb3.ms to bx610_b6c3_uv_mc/imod2d_b6c3_bb3.fits
imod3d@../data/bx610/alma/2015.1.00250.S/bb3.ms
write reference model image: 
    imod3d@../data/bx610/alma/2015.1.00250.S/bb3.ms to bx610_b6c3_uv_mc/imod3d_b6c3_bb3.fits
pbeam@../data/bx610/alma/2015.1.00250.S/bb3.ms
write reference model image: 
    pbeam@../data/bx610/alma/2015.1.00250.S/bb3.ms to bx610_b6c3_uv_mc/pbeam_b6c3_bb3.fits
write reference model profile: 
    imod3d_prof@h2o@../data/bx610/alma/2015.1.00250.S/bb3.ms to bx6

In [54]:
blobs.item()['npar']

NameError: name 'blobs' is not defined

In [ ]:
gmake.fit_iterate(fit_dct,sampler,nstep=inp_dct['optimize']['niter'])

In [ ]:
gmake.fit_analyze(inpfile)

In [40]:
import astropy.units as u

In [41]:
x=1.0*u.deg

In [43]:
x.to(unit=u.FITS)

AttributeError: module 'astropy.units' has no attribute 'FITS'